In [250]:
import numpy as np

from connect4.board import Piece, Board

grid = np.full((6, 7), Piece(' '), dtype=Piece)

def add_piece(grid, row, col, piece: Piece) -> None:
    grid[row,col] = piece

piece = Piece('Y')

In [12]:
%%timeit
add_piece(grid, 3, 4, piece)

741 ns ± 57.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [16]:
from enum import Enum, auto

class Piece2(Enum):
    EMPTY = auto()
    RED = auto()
    YELLOW = auto()

grid = np.full((6, 7), Piece2(Piece2.EMPTY), dtype=Piece)

def add_piece(grid, row, col, piece: Piece2) -> None:
    grid[row,col] = piece

piece = Piece2(Piece2.RED)

In [17]:
%%timeit
add_piece(grid, 3, 4, piece)

624 ns ± 76.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [247]:
def _is_diagonal_winner(pos, player_pieces: np.ndarray) -> bool:
        """Checks if player_pieces contains a vertical or horizontal win."""
        r, c  = pos
        L = 4
        C, R = 7-1, 6-1
        topleft = min(r,c)
        bottomright = min(R-r,C-c)
        topright = min(r,C-c)
        bottomleft = min(R-r,c)
        rng1 = range(max(0,L-bottomright-1),min(L-1,topleft)+1)
        rng2 = range(max(0,L-topright-1),min(L-1,bottomleft)+1)
        for i in rng1:
            print([(r - i + x, c - i + x) for x in range(L)])
            if all(player_pieces[r + i + x -1][c + i + x -1] for x in range(L)):
                    return True
        print(rng2)
        for i in rng2:
            print([(r + i - x, c - i + x) for x in range(L)])
            #if all(player_pieces[r - i - x -1][c + i + x -1] for x in range(L)):
            #        return True

In [248]:
b = np.zeros((6,7),dtype=int)

pos = (3,3)

r, c  = pos
#idx = tuple(x-1 for x in pos)
#b[idx] = 1
b[pos] =1

_is_diagonal_winner(pos, b)

print(b)

3 2 3 2
range(1, 4)
[(2, 2), (3, 3), (4, 4), (5, 5)]
[(1, 1), (2, 2), (3, 3), (4, 4)]
[(0, 0), (1, 1), (2, 2), (3, 3)]
range(0, 3)
[(3, 3), (2, 4), (1, 5), (0, 6)]
[(4, 2), (3, 3), (2, 4), (1, 5)]
[(5, 1), (4, 2), (3, 3), (2, 4)]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]


In [67]:
L = 4
C, R = 7, 6
topleft = min(L-1,r,c)
bottomright = min(L-1,R-r,C-c)
topright = min(L-1,r,C-c)
bottomleft = min(L-1,R-r,c)
print(topleft,bottomleft,bottomright,topright)
range(-topleft, bottomright-L)

3 2 2 2


range(-3, -2)

In [ ]:
def _is_winner(self, player_pieces:np.ndarray) -> bool:
        r, c  = self.last_pos
        L = self.four
        C, R = self.n_c - 1, self.n_r - 1

        # check diagonals first, return if win is found
        bottomright = min(R - r, C - c)
        topright = min(r, C - c)
        for i in range(max(0, L- bottomright - 1), min(L - 1, r, c) + 1):  #  topleft = min(r,c)
            if all(player_pieces[r - i + x][c - i + x] for x in range(L)):
                    return True

        for i in range(max(0, L - topright - 1), min(L - 1, R - r, c) + 1):  #  bottomleft = min(R-r,c)
            if all(player_pieces[r + i - x][c - i + x] for x in range(L)):
                    return True

In [251]:
from collections import namedtuple 

class BoardOld: 
    LENGTH = 7 
    HEIGHT = 6 
    WIN_L = 4 
 
    WinState = namedtuple('WinState', ['is_ended', 'winner']) 
 
    def __init__(self) -> None: 
        self.grid = np.full((self.HEIGHT, self.LENGTH), Piece.EMPTY, dtype=Piece) 
        self.input_idx = np.zeros(self.LENGTH, dtype=int) 
        self.last_pos = (0,0) 
 
    def add_piece(self, col, piece: Piece) -> None: 
        if self.input_idx[col] >= self.HEIGHT: 
            raise ValueError(f"Can't put piece in column {col}.") 
        else: 
            self.grid[self.input_idx[col],col] = piece 
            self.last_pos = (self.input_idx[col], col) 
            self.input_idx[col] += 1 
 
    @property 
    def valid_moves(self) -> np.ndarray: 
        return self.input_idx < self.HEIGHT 
 
    def _is_straight_winner(self, player_pieces: np.ndarray) -> bool: 
        """Checks if player_pieces contains a vertical or horizontal win.""" 
        run_lengths = [player_pieces[:, i:i+self.WIN_L].sum(axis=1) 
                       for i in range(len(player_pieces) - self.WIN_L + 1)] 
        return any(x.max() >= self.WIN_L for x in run_lengths) 

    def _is_diagonal_winner(self, player_pieces: np.ndarray) -> bool: 
        """Checks if player_pieces contains a diagonal win.""" 
        L = len(player_pieces) 
        for i in range(L - self.WIN_L + 1): 
            for j in range(L - self.WIN_L + 1): 
                if all(player_pieces[i + x][j + x] for x in range(self.WIN_L)): 
                    return True 
            for j in range(self.WIN_L - 1, L): 
                if all(player_pieces[i + x][j - x] for x in range(self.WIN_L)): 
                    return True 
        return False 
 
    def get_win_state(self, player:Piece) -> WinState: 
        player_pieces = self.grid == player 
        # Check rows & columns for win 
        if (self._is_straight_winner(player_pieces) or 
            self._is_straight_winner(player_pieces.T) or 
            self._is_diagonal_winner(player_pieces)): 
            return self.WinState(True, player) 
 
        # Draw 
        if not self.valid_moves.any(): 
            return self.WinState(True, Piece.EMPTY) 
 
        # Game is not ended yet. 
        return self.WinState(False, Piece.EMPTY)

In [252]:
from connect4.players import Rand

In [258]:
%%timeit -n 1000
b = Board()
now, prev = 0, 1
player = (Rand(Piece('R')), Rand(Piece('Y')))
while not b.get_win_state(player[prev].color).is_ended:
    column = player[now].get_action(b)
    b.add_piece(int(column), player[now].color)
    prev = now
    now = 1*(1-now)

2.9 ms ± 302 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [259]:
%%timeit -n 1000
b = BoardOld()
now, prev = 0, 1
player = (Rand(Piece('R')), Rand(Piece('Y')))
while not b.get_win_state(player[prev].color).is_ended:
    column = player[now].get_action(b)
    b.add_piece(int(column), player[now].color)
    prev = now
    now = 1*(1-now)

3.2 ms ± 288 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
